In [1]:
import findspark
findspark.init()
findspark.add_packages('mysql:mysql-connector-java:8.0.11')

In [2]:
from pyspark.sql.types import StringType

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("writemysql") \
.config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
    .config("spark.jars.packages", "mysql:mysql-connector-java:8.0.17") \
.getOrCreate()

In [5]:
import mysql.connector

In [6]:
conn = mysql.connector.connect(user = 'root', database='dataengg',
                               password='admin',
                               host="127.0.0.1",
                               )

In [6]:
# cursor = conn.cursor()

In [7]:
# sqlquery1 = "drop table if exists persontransformed"

In [8]:
# cursor.execute(sqlquery1)

In [9]:
# conn.close()

In [7]:
personrawdf = spark.read \
        .format("jdbc") \
        .option("driver","com.mysql.cj.jdbc.Driver") \
        .option("url", "jdbc:mysql://localhost:3306/dataengg") \
        .option("dbtable", "dataengg.personraw") \
        .option("user", "root") \
        .option("password", "admin")  \
        .load()
        

In [8]:
personrawdf.show()

+---+--------------+---+------+----+-----------+-------------------+
| id|          name|age|gender|rank|active_flag|           currtime|
+---+--------------+---+------+----+-----------+-------------------+
|  1| Roger Federer| 38|     M|   1|          Y|2023-01-07 19:03:53|
|  2|  Rafael Nadal| 34|     M|   2|          Y|2023-01-07 19:03:53|
|  3|Novak Djokovic| 32|     M|   3|          Y|2023-01-07 19:03:53|
|  4|   Andy Murray| 31|     M|   4|          Y|2023-01-07 19:03:53|
|  5|   Mario Cilic| 30|     M|   5|          Y|2023-01-07 19:03:53|
|  6|Carlos Alcaraz| 19|     M|   6|          Y|2023-01-09 20:08:59|
|  7|   Casper Rudd| 24|     M|   7|          Y|2023-01-09 20:08:59|
+---+--------------+---+------+----+-----------+-------------------+



In [9]:
persontransformeddf = spark.read \
        .format("jdbc") \
        .option("driver","com.mysql.cj.jdbc.Driver") \
        .option("url", "jdbc:mysql://localhost:3306/dataengg") \
        .option("dbtable", "dataengg.persontransformed") \
        .option("user", "root") \
        .option("password", "admin")  \
        .load()

In [10]:
persontransformeddf.show()

+---+--------------+---+------+----+-----------+-------------------+
| id|          name|age|gender|rank|active_flag|           currtime|
+---+--------------+---+------+----+-----------+-------------------+
|  1| Roger Federer| 38|     M|   1|          Y|2023-01-07 19:03:53|
|  2|  Rafael Nadal| 34|     M|   2|          Y|2023-01-07 19:03:53|
|  3|Novak Djokovic| 32|     M|   3|          Y|2023-01-07 19:03:53|
|  4|   Andy Murray| 31|     M|   4|          Y|2023-01-07 19:03:53|
|  5|   Mario Cilic| 30|     M|   5|          Y|2023-01-07 19:03:53|
+---+--------------+---+------+----+-----------+-------------------+



In [19]:
#select(*) during initial load
#persontransformeddf = personrawdf.select("*")
# persontransformeddf = personrawdf.filter(personrawdf.select(['currtime']) > personrawdf.agg({'currtime': 'max'}))
lastingestedtimedf = persontransformeddf.agg({'currtime': 'max'})
lastingestedtimedf.show()
# lastingestedtimestr = str(lastingestedtime)
strlastingested = str(lastingestedtimedf.first()['max(currtime)'])
strlastingested

+-------------------+
|      max(currtime)|
+-------------------+
|2023-01-07 19:03:53|
+-------------------+



'2023-01-07 19:03:53'

In [65]:
lastingestedtime.show()

+-------------------+
|      max(currtime)|
+-------------------+
|2023-01-07 19:03:53|
+-------------------+



In [13]:
persontransformeddf.select(['currtime']).show()

+-------------------+
|           currtime|
+-------------------+
|2023-01-07 19:03:53|
|2023-01-07 19:03:53|
|2023-01-07 19:03:53|
|2023-01-07 19:03:53|
|2023-01-07 19:03:53|
+-------------------+



In [20]:
persontransformeddf = personrawdf.filter(personrawdf['currtime'] > strlastingested)

In [24]:
persontransformeddf.show()

+---+--------------+---+------+----+-----------+-------------------+
| id|          name|age|gender|rank|active_flag|           currtime|
+---+--------------+---+------+----+-----------+-------------------+
|  6|Carlos Alcaraz| 19|     M|   6|          Y|2023-01-09 20:08:59|
|  7|   Casper Rudd| 24|     M|   7|          Y|2023-01-09 20:08:59|
+---+--------------+---+------+----+-----------+-------------------+



In [31]:
persontransformeddf.write.mode("append") \
.format("jdbc") \
.options(driver = "com.mysql.cj.jdbc.Driver",user ="root",password = "admin",dbtable= "persontransformed",url = "jdbc:mysql://localhost:3306/dataengg") \
.save()

In [30]:
#one time write to target table
# persontransformeddf.write \
# .format("jdbc") \
# .option("driver","com.mysql.cj.jdbc.Driver") \
# .option("url", "jdbc:mysql://localhost:3306/dataengg") \
# .option("dbtable", "persontransformed") \
# .option("user", "root") \
# .option("password", "admin") \
# .option("mode","append") 
# # .save()

In [ ]:
#one time write to target table
lastingestedtimestr.write \
.format("jdbc") \
.option("driver","com.mysql.cj.jdbc.Driver") \
.option("url", "jdbc:mysql://localhost:3306/dataengg") \
.option("dbtable", "lastingesteddate") \
.option("user", "root") \
.option("password", "admin") \
.save()